<a href="https://colab.research.google.com/github/taerri/meal-kit-analysis/blob/main/02_%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88_%EB%B0%80%ED%82%A4%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC_Mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

konlpy 설치

In [1]:
pip install konlpy

Mecab 설치

In [2]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141817 sha256=b9361d70cdcaf42a4cba861b7dc1f41a63de9be302f2cda77ee59e0e42b21716
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [3]:
import konlpy
import MeCab
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
import re

In [4]:
df=pd.read_csv("크롤링 결과.csv")
df.head()

,Unnamed: 0,Date,Title,Content
0,0,2017-12-27,"연말연시 ‘홈파티족’ 확산, 홈파티 ‘잇츠온’ 간편식 직접 체험해보니",요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1,1,2017-12-27,식물성·간편식·슈퍼곡물에 빠졌던 2017년,식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2,2,2017-12-27,"올해만 10여개 브랜드 출시, 가정간편식 '춘추전국시대'","야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나..."
3,3,2017-12-26,유통업계 '밀 솔루션' 경쟁 본격화,유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4,4,2017-12-19,"해먹남녀, GS리테일 ‘심플리쿡’ 판매 진행 “칼질 없이 일품요리 즐겨”",푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...


In [5]:
len(df)

5630

In [6]:
#본문 리스트 만들기
word_list=[]
word_list=df['Content']
word_list=word_list.reset_index(drop=True)

In [7]:
#기사중복 삭제
df_drop=df.drop_duplicates(['Date','Title','Content'], keep='first')

In [8]:
#index reset
df=df_drop.reset_index(drop=True)

In [9]:
#행,열 최대로 보기
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

In [10]:
df.head()
len(df)

5507

In [11]:
#기사 본문만 가져오기
content=df["Content"]
content.head()

0    요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1    식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2    야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나...
3    유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4    푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...
Name: Content, dtype: object

In [12]:
#None값 제거
drop_row = content.dropna(axis=0)
df=pd.DataFrame([drop_row])
df= df.transpose()

In [13]:
#단어 교체
content=df['Content'].map(lambda x: str(x).replace('밀 키트','밀키트'))

# **명사 추출**

In [14]:
mecab=Mecab()

In [15]:
#명사 추출
from tqdm import tqdm
nouns = [] 
for x in tqdm(content):
    try:
        nouns.append(mecab.nouns(x))
    except:
        nouns.append(["None"])

100%|██████████| 5506/5506 [00:20<00:00, 274.27it/s]


In [16]:
#명사 추출 결과값 열에 추가
konlpy_df= df.copy()
konlpy_df["ContentNoun"] = nouns

In [17]:
#json 파일로 저장
konlpy_df.to_json("mealkit_전처리.json",orient="table")

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import pickle
import re

In [19]:
corpus = pd.read_json("mealkit_전처리.json",orient="table")

# **단어 교체**

In [20]:
#단어 교체(11번 -> 11번가)
def change(text):
    text=','.join(text)
    text_list=text.split(',')
    result=[]
    for c in text_list:
        if c=='11번':
            c=c.replace('11번','11번가')
        if c=='인공지능':
            c=c.replace('인공지능','AI')
        result.append(c)
    result=','.join(result)
    return result

In [21]:
corpus['ContentNoun']=corpus['ContentNoun'].map(lambda x: change(x))

In [22]:
tokenized_doc = list(corpus["ContentNoun"])

# **단어 한글자 이하 제거**

In [23]:
tokenized_doc = [n for n in tokenized_doc if len(n)>1]

# **불용어 처리**

## 불용어 제거
- 사이트 이용
    - [사이트1](https://bab2min.tistory.com/544)
    - [사이트2](https://www.ranks.nl/stopwords/korean)
- 직접 제거
    - ./삭제 단어.xlsx

In [24]:
#직접 제거
stopword1 = pd.read_csv("./불용어 처리.txt",header=None,sep="\t")
stopword1 = list(stopword1[0])
len(stopword1)

139

In [25]:
stopword2 = pd.read_csv("한국어불용어100.txt",header=None,sep="\t")
stopword2 = list(stopword2[0])
len(stopword2)

100

In [26]:
stopword3 = pd.read_csv("한국어불용어100_2.txt",header=None)
stopword3 = list(stopword3[0])
len(stopword3)

675

In [27]:
stopword = list(set(stopword1 + stopword2 + stopword3))
print("불용어 개수 :",len(stopword))

불용어 개수 : 803


In [28]:
tokenized_doc = [[x for x in y if x not in stopword] for y in tokenized_doc]

In [29]:
corpus.to_json("mealkit_전처리.json",orient="table")

In [30]:
file=corpus[["ContentNoun"]]
file.columns=['키워드']
file.to_csv("단어(전처리 후).csv",encoding="utf-8-sig")

In [31]:
file.head()

,키워드
0,"요리,재미,맛,일품,분위기,덤,거리,인파,시간,기본,식당,카페,대기,시간,고가,크리..."
1,"식품,낭비,제로,웨이스트,발달,이국,맛,스닉,푸드,등,식품,트렌드,가지,건강,삶,중..."
2,"야구르트,아줌마,배달,간,편식,백화점,프리미엄,간,편식,편의점,간,편식,등,간,편식..."
3,"유통,업계,소비자,라이프,스타일,변화,끼,식사,가능,제품,서비스,제공,밀,솔루션,시..."
4,"푸드,콘텐츠,커머스,플랫,폼,해먹,남녀,운영,바이탈,힌트,코리아,대표,정지웅,리,테..."
